In [1]:
#!pip install eeglib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from eeglib.helpers import Helper
from eeglib.eeg import EEG

In [2]:
def df_to_eeg_helper(df, columns, window_size, high_pass=50.0, low_pass=1.0, normalize=False):
    data = df[columns].copy().to_numpy().transpose()
    helper = Helper(
        data, 
        sampleRate=200, 
        names=columns, 
        windowSize=window_size,
        highpass=high_pass,
        lowpass=low_pass,
        normalize=normalize
    )
    return helper

In [ ]:
eeg_df = pd.read_csv("train_eeg/175001942_filtered.csv", index_col=0)

In [4]:
helper = df_to_eeg_helper(eeg_df.iloc[4000:6000, :], list(eeg_df.columns), window_size=2000, low_pass=None, high_pass=None)

In [5]:
EEG = [eeg for eeg in helper][0]

In [6]:
EEG

In [31]:
bp = EEG.bandPower()

In [9]:
EEG.PSD().shape

(18, 501)

In [ ]:
## Adapted from https://github.com/adrianaleal/eeg-preictal-identification-epilepsy/blob/main/1_extract_eeg_features/FunctionsFeatures/LinearUnivariate/spectral_edge.m
def spectral_edge_frequency(frequency, psd):
    ## Input:
    ## frequency - vector with frequency at which PSD was computed by EEG.PSD()
    ## psd - vector with PSD values from EEG.PSD()
    
    ## Output:
    ## sef - Spectral edge freqeuncy

    # get the indexes of the 0–40 Hz frequency band
    indices = np.argwhere(frequency<=40)

    # get the 0–40 Hz frequency and power vectors
    frequency_band = frequency[indices]
    power_band = psd[indices]

    # get the total power in that band
    total_spectral_power_band = sum(power_band)

    # define the percentage of overall power
    power_threshold = 50

    # get the the corresponding value of power
    spectral_power_threshold = total_spectral_power_band*(power_threshold/100)

    # get the cumulative power
    cumulative_power_band = np.cumsum(power_band)

    # get the the corresponding value of frequency and power
    ind_spectral_edge = np.argwhere(cumulative_power_band>=spectral_power_threshold)[0]
    spectral_edge_frequency = frequency_band[ind_spectral_edge]

    return spectral_edge_frequency[0]

In [39]:
def get_features(eeg):
    
    # Absolute Band Power values in db
    bp = eeg.bandPower(bands={'alpha': (8, 12), 'beta': (13, 20), 'delta': (1, 4), 'theta': (4, 7)})
    # Split band power and convert to db
    bp_alpha = [10.0 * np.log10(ch["alpha"]) for ch in bp]
    bp_beta = [10.0 * np.log10(ch["beta"]) for ch in bp]
    bp_delta = [10.0 * np.log10(ch["delta"]) for ch in bp]
    bp_theta = [10.0 * np.log10(ch["theta"]) for ch in bp]

    # Relative Band Power values
    bp = eeg.bandPower(bands={'alpha': (8, 12), 'beta': (13, 20), 'delta': (1, 4), 'theta': (4, 7)},
                       normalize=True)
    # Split band power and convert to db
    bp_rel_alpha = [ch["alpha"] for ch in bp]
    bp_rel_beta = [ch["beta"] for ch in bp]
    bp_rel_delta = [ch["delta"] for ch in bp]
    bp_rel_theta = [ch["theta"] for ch in bp]

    # Spectral edge frequency
    freq_psd = EEG.PSD(retFrequencies=True)
    sef = []
    # iterating over each channel
    for i in range(freq_psd.shape[0]):
        sef.extend(spectral_edge_frequency(freq_psd[i,0,:], freq_psd[i,1,:]))
    sef = np.asarray(sef)

    hfd = list(eeg.HFD())
    hjorth_mobility = list(eeg.hjorthMobility())
    hjorth_complexity = list(eeg.hjorthComplexity())
    dfa = list(eeg.DFA())
    sampEn = list(eeg.sampEn())

    df_as_dict = {}
    #print(bp_alpha)
    for chan_idx, col_name in enumerate(helper.names):
        df_as_dict[f"{col_name}.abs_bp_alpha"] = [bp_alpha[chan_idx]]
        df_as_dict[f"{col_name}.abs_bp_beta"] = [bp_beta[chan_idx]]
        df_as_dict[f"{col_name}.abs_bp_delta"] = [bp_delta[chan_idx]]
        df_as_dict[f"{col_name}.abs_bp_theta"] = [bp_theta[chan_idx]]
        
        df_as_dict[f"{col_name}.rel_bp_alpha"] = [bp_rel_alpha[chan_idx]]
        df_as_dict[f"{col_name}.rel_bp_beta"] = [bp_rel_beta[chan_idx]]
        df_as_dict[f"{col_name}.rel_bp_delta"] = [bp_rel_delta[chan_idx]]
        df_as_dict[f"{col_name}.rel_bp_theta"] = [bp_rel_theta[chan_idx]]
        
        #[delta/alpha, delta/beta, delta/theta, theta/alpha, theta/beta, alpha/beta, beta/(alpha+theta), and theta/(alpha+beta)
        # Ratios calculated using relative bp because of unit conversion to db applied above to abs bp 
        df_as_dict[f"{col_name}.bp_delta_alpha"] = [bp_rel_delta[chan_idx]/bp_rel_alpha[chan_idx]]
        df_as_dict[f"{col_name}.bp_delta_beta"] = [bp_rel_delta[chan_idx]/bp_rel_beta[chan_idx]]
        df_as_dict[f"{col_name}.bp_delta_theta"] = [bp_rel_delta[chan_idx]/bp_rel_theta[chan_idx]]
        df_as_dict[f"{col_name}.bp_theta_alpha"] = [bp_rel_theta[chan_idx]/bp_rel_alpha[chan_idx]]
        df_as_dict[f"{col_name}.bp_theta_beta"] = [bp_rel_theta[chan_idx]/bp_rel_beta[chan_idx]]
        df_as_dict[f"{col_name}.bp_alpha_beta"] = [bp_rel_alpha[chan_idx]/bp_rel_beta[chan_idx]]
        df_as_dict[f"{col_name}.bp_beta_alpha+theta"] = [bp_rel_beta[chan_idx]/(bp_rel_alpha[chan_idx]+bp_rel_theta[chan_idx])]
        df_as_dict[f"{col_name}.bp_theta_alpha+beta"] = [bp_rel_theta[chan_idx]/(bp_rel_alpha[chan_idx]+bp_rel_beta[chan_idx])]

        df_as_dict[f"{col_name}.spectral_edge_freq"] = [sef[chan_idx]]
        df_as_dict[f"{col_name}.spectral_edge_freq"] = [sef[chan_idx]]
        df_as_dict[f"{col_name}.spectral_edge_freq"] = [sef[chan_idx]]
        df_as_dict[f"{col_name}.spectral_edge_freq"] = [sef[chan_idx]]
        
        df_as_dict[f"{col_name}.hfd"] = [hfd[chan_idx]]
        df_as_dict[f"{col_name}.hjorth_mobility"] = [hjorth_mobility[chan_idx]]
        df_as_dict[f"{col_name}.hjorth_complexity"] = [hjorth_complexity[chan_idx]]
        df_as_dict[f"{col_name}.dfa"] = [dfa[chan_idx]]
        df_as_dict[f"{col_name}.samp_en"] =[sampEn[chan_idx]]

    df = pd.DataFrame.from_dict(df_as_dict)
    return df.copy()

In [40]:
features = get_features(EEG)

In [41]:
features

,0.abs_bp_alpha,0.abs_bp_beta,0.abs_bp_delta,0.abs_bp_theta,0.rel_bp_alpha,0.rel_bp_beta,0.rel_bp_delta,0.rel_bp_theta,0.bp_delta_alpha,0.bp_delta_beta,...,15.bp_theta_beta,15.bp_alpha_beta,15.bp_beta_alpha+theta,15.bp_theta_alpha+beta,15.spectral_edge_freq,15.hfd,15.hjorth_mobility,15.hjorth_complexity,15.dfa,15.samp_en
0,26.847741,27.756743,33.301725,28.442888,0.058249,0.07181,0.257445,0.084101,4.419757,3.585075,...,1.425772,1.872753,0.303166,0.496309,4.4,2.069853,0.293823,2.259126,1.050634,0.928106
